In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Inspect model interactively

In this notebook, we illustrate how to interactively inspect the {class}`.HelicityModel` that was created in {doc}`amplitude`. We do this with the excellent {doc}`mpl-interactions <mpl_interactions:index>` package.

First, we recreate {class}`.HelicityModel`. We could also have used {mod}`pickle` to {func}`~pickle.load` the {class}`.HelicityModel` that we created in {doc}`amplitude`, but the cell below allows running this notebook independently.

In [ ]:
import qrules as q

from ampform import get_builder
from ampform.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

result = q.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)", "f(0)(1500)"],
    allowed_interaction_types=["strong", "EM"],
    formalism_type="canonical-helicity",
)
model_builder = get_builder(result)
initial_state_particle = result.get_initial_state()[0]
model_builder.set_dynamics(initial_state_particle, create_non_dynamic_with_ff)
for name in result.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.generate()

In this case, {ref}`as we saw <usage/amplitude:Mathematical formula>`, the overall model contains just one intensity term $I = |\sum_i A_i|^2$, with $\sum_i A_i$ some coherently sum of amplitudes. We can get the amplitude $\sum_i A_i$ as follows:

In [ ]:
import sympy as sp

amplitude = model.expression.args[0].args[0].args[0]
assert isinstance(amplitude, sp.Add)

Replace some of the boring parameters with the provided {attr}`~.HelicityModel.parameter_defaults`:

In [ ]:
amplitude = amplitude.doit().subs(
    {
        s: v
        for s, v in model.parameter_defaults.items()
        if not s.name.startswith("Gamma") and not s.name.startswith("m_f(0)")
    }
)
symbols = sorted(amplitude.free_symbols, key=lambda s: s.name)
symbols

Identify the symbol over which the amplitude is to be plotted. The remaining symbols will be turned into slider parameters

In [ ]:
plot_variable = sp.Symbol("m_12", real=True)
slider_variables = symbols
slider_variables.remove(plot_variable)

Next, use them to {func}`~sympy.utilities.lambdify.lambdify` the expression:

In [ ]:
np_amplitude = sp.lambdify(
    (plot_variable, *slider_variables),
    amplitude,
    "numpy",
)

The function arguments are 'dummified' if the {class}`~sympy.core.symbol.Symbol` name is not a valid name for a Python variable. We therefore need some mapping between the {class}`~sympy.core.symbol.Symbol` and their 'dummified' name. This can be created with {func}`inspect.signature`:

In [ ]:
import inspect

variable_names = list(
    map(lambda s: s.name, (plot_variable, *slider_variables))
)
arg_names = list(inspect.signature(np_amplitude).parameters)
variable_to_arg = dict(zip(variable_names, arg_names))
variable_to_arg

Now we are ready to use {doc}`mpl-interactions <mpl_interactions:index>` to investigate the behavior of the amplitude model. First, we define some functions that formulate what we want to plot:

In [ ]:
import numpy as np


def intensity(plot_variable, **kwargs):
    values = np_amplitude(plot_variable, **kwargs)
    return np.abs(values) ** 2


def argand(**kwargs):
    values = np_amplitude(plot_domain, **kwargs)
    argand = np.array([values.real, values.imag])
    return argand.T

Next, we need to define the domain over which to plot, as well as a domains for the sliders that are to represent the parameter values:

In [ ]:
plot_domain = np.linspace(0.2, 2.5, num=400)
slider_domains = {
    "m_f(0)(980)": np.linspace(0.3, 1.8, 100),
    "m_f(0)(1500)": np.linspace(0.3, 1.8, 100),
    "Gamma_f(0)(980)": np.linspace(0.01, 1, 100),
    "Gamma_f(0)(1500)": np.linspace(0.01, 1, 100),
    "m_1": np.linspace(0.01, 1, 100),
    "m_2": np.linspace(0.01, 1, 100),
    "phi_1+2": np.arange(0, 2 * np.pi, step=np.pi / 40),
    "theta_1+2": np.arange(-np.pi, np.pi, step=np.pi / 40),
}

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)
if STATIC_WEB_PAGE:
    # Concatenate flipped domain for reverse animation
    domain = np.linspace(0.8, 2.2, 100)
    domain = np.concatenate((domain, np.flip(domain[1:])))
    slider_domains["m_f(0)(980)"] = domain

Similarly, it is nice to supply some starting values for the sliders:

In [ ]:
pdg = q.load_pdg()
starting_values = {
    symbol.name: value
    for symbol, value in model.parameter_defaults.items()
    if symbol.name in variable_names
}
starting_values["m_1"] = pdg["pi0"].mass
starting_values["m_2"] = pdg["pi0"].mass
starting_values["phi_1+2"] = 0
starting_values["theta_1+2"] = 0

Finally, use {doc}`mpl-interactions <mpl_interactions:index>` to plot the functions defined above with regard to the parameter values:

:::{margin}

Interactive {mod}`~matplotlib.widgets` do not render well on web pages, so run this notebook in Google Colab or in Jupyter Lab to see the full power of {doc}`mpl-interactions <mpl_interactions:index>`!

:::

In [ ]:
%matplotlib widget
import warnings

import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt

warnings.filterwarnings("ignore")

# Create figure
fig, axes = plt.subplots(
    1, 2, figsize=0.9 * np.array((8, 3.8)), tight_layout=True
)
fig.suptitle(R"$J/\psi \to \gamma f_0, f_0 \to \pi^0\pi^0$")
ax_intensity, ax_argand = axes
m_label = R"$m_{\pi^0\pi^0}$ (GeV)"
ax_intensity.set_xlabel(m_label)
ax_intensity.set_ylabel("$I = |A|^2$")
ax_argand.set_xlabel("Re($A$)")
ax_argand.set_ylabel("Im($A$)")

# Fill plots
parameters = {
    variable_to_arg[var]: domain for var, domain in slider_domains.items()
}
controls = iplt.plot(
    plot_domain,
    intensity,
    **parameters,
    slider_formats={slider: "{:.3f}" for slider in arg_names},
    ylim="auto",
    ax=ax_intensity,
)
iplt.scatter(
    argand,
    controls=controls,
    xlim="auto",
    ylim="auto",
    parametric=True,
    c=plot_domain,
    s=1,
    ax=ax_argand,
)
plt.colorbar(label=m_label, ax=ax_argand, aspect=30, pad=0.01)
plt.winter()


# Rename sliders
for var, arg in variable_to_arg.items():
    if arg == plot_variable.name:
        continue
    latex = sp.latex(sp.Symbol(var))
    controls.controls[arg].children[0].description = f"${latex}$"


# Set initial values for the sliders
def find_index(array, value):
    for i in range(len(array)):
        if array[i] >= value:
            return i
    return 0


for var_name, start_value in starting_values.items():
    arg = variable_to_arg[var_name]
    domain = slider_domains[var_name]
    slider_position = find_index(domain, start_value)
    controls.controls[arg].children[0].value = slider_position

In [ ]:
# Export for Read the Docs
if STATIC_WEB_PAGE:
    output_path = "animation.gif"
    arg_name = variable_to_arg["m_f(0)(980)"]
    controls.save_animation(output_path, fig, arg_name, fps=25)

:::{margin}

This figure is an animation over **just one of the parameters**. Run the notebook itself to play around with all parameters!

:::

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import Image

    with open(output_path, "rb") as f:
        display(Image(data=f.read(), format="png"))